# The litdjango CLI


In order to adapt Django to a literate programming style using Jupyter notebooks, we need to
reimplement any builtin `django-admin` and `manage.py` cli commands which produce `.py` modules
into commands which produce `.ipynb` files.

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from collections import defaultdict
import sys

import django
from django.conf import settings
from django.core.management import ManagementUtility, get_commands
from django.core.management.base import BaseCommand
from django.core.management.color import color_style
from django.core.management.commands.startapp import Command as StartApp
from django.core.management.commands.startproject import Command as StartProject
from nbdev.export import nb_export
from pathlib import Path

from litdjango.utils import LITDJANGO_ROOT, get_project_config, get_project_root

## Subcommands

#### `litdjango startproject`

In [ ]:
#| export
class StartProject(StartProject):
    rewrite_template_suffixes = (('.py-tpl', '.py'), (('.ipynb-tpl', '.ipynb')))

    def handle(self, **options):
        options["template"] = str(LITDJANGO_ROOT / "templates" / "project_template")
        options["extensions"] = ['py', 'txt', 'ipynb', 'ini']
        super().handle(**options)

In [ ]:
import os
from pathlib import Path
import subprocess
import shutil

In [ ]:
cwd = Path.cwd()
cwd

Path('/home/aaron/projects/litdjango/litdjango/nbs')

In [ ]:
project_name = "ldsp"
project_dir = cwd.parent.parent / "litdjango_test" / project_name # TODO refactor this into temp directory

In [ ]:
if project_dir.exists():
    shutil.rmtree(project_dir)
project_dir.mkdir(exist_ok=True)
os.chdir(project_dir)
subprocess.check_call(["litdjango", "startproject", project_name, "."])
subprocess.check_call(["tree", "-I", "__pycache__"])
os.chdir(cwd)
shutil.rmtree(project_dir)

.
├── ldsp
├── nbs
│   ├── config
│   │   ├── asgi.ipynb
│   │   ├── settings.ipynb
│   │   ├── urls.ipynb
│   │   └── wsgi.ipynb
│   └── manage.ipynb
└── settings.ini

3 directories, 6 files


#### `litdjango export`

In [ ]:
#| export
class Export(BaseCommand):
    help = "Export .ipynb notebooks to .py modules"
    requires_system_checks = []
    
    def handle(self, *args, **options):
        if not settings.configured:
            settings.configure()
            django.setup()
        self.stdout.write("Exporting...")
        path = Path.cwd()
        cfg = get_project_config(path)
        project_root = get_project_root(path)
        lib_path = project_root / cfg["lib_name"]
        nbs_path = project_root / cfg["nbs_path"]
        notebooks = [nb for nb in nbs_path.glob("**/*.ipynb")]
        for nb in notebooks: nb_export(nb, lib_path)

In [ ]:
if project_dir.exists():
    shutil.rmtree(project_dir)
project_dir = cwd.parent.parent / "litdjango_test/ldsp"
project_dir.mkdir(exist_ok=True)
os.chdir(project_dir)
subprocess.check_call(["litdjango", "startproject", "ldsp", "."])
subprocess.check_call(["litdjango", "export"])
subprocess.check_call(["tree", "-I", "__pycache__", str(project_dir)])
subprocess.check_call(["python", f"{project_name}/manage.py", "check"])
os.chdir(cwd)
shutil.rmtree(project_dir)

Exporting...
/home/aaron/projects/litdjango/litdjango_test/ldsp
├── ldsp
│   ├── config
│   │   ├── asgi.py
│   │   ├── settings.py
│   │   ├── urls.py
│   │   └── wsgi.py
│   └── manage.py
├── nbs
│   ├── config
│   │   ├── asgi.ipynb
│   │   ├── settings.ipynb
│   │   ├── urls.ipynb
│   │   └── wsgi.ipynb
│   └── manage.ipynb
└── settings.ini

4 directories, 11 files
System check identified no issues (0 silenced).


#### `litdjango startapp`

In [ ]:
#| export
class StartApp(StartApp):
    pass

## Register subcommands with the cli

In [ ]:
#| exporti
lit_commands = {
    "startapp": StartApp(), # overrides django default
    "export": Export(), # new command
    "startproject": StartProject() #overrides django default
}

In [ ]:
#| export
class LitManagementUtility(ManagementUtility):
    """The litdjango cli is an instance of this class
    
    We use the lit_commands dict as a container for new litdjango commands, or 
    overridden django commands, and fallback to the default django command for anything
    else.
    """
    def __init__(self, argv=None):
        super().__init__(argv)
        
    def main_help_text(self, commands_only=False):
        """Return the script's main help text, as a string."""
        if commands_only:
            usage = sorted(set(get_commands()) | set(lit_commands))  # overwrite django
        else:
            usage = [
                "",
                "Type '%s help <subcommand>' for help on a specific subcommand." % self.prog_name,
                "",
                "Available subcommands:",           
                ]
            commands_dict = defaultdict(lambda: []) 
            commands_dict["litdjango"] = sorted(lit_commands.keys())
            for name, app in get_commands().items():
                if app == "django.core":
                    if name in lit_commands.keys():
                        continue
                    else:
                        app = "django"
                else:
                    app = app.rpartition(".")[-1]
                commands_dict[app].append(name)
            style = color_style()
            for app in sorted(commands_dict):
                usage.append("")
                usage.append(style.NOTICE("[%s]" % app))
                for name in sorted(commands_dict[app]):
                    usage.append("    %s" % name)
            # Output an extra note if settings are not properly configured
            if self.settings_exception is not None:
                usage.append(
                    style.NOTICE(
                        "Note that only Django core commands are listed "
                        "as settings are not properly configured (error: %s)."
                        % self.settings_exception
                    )
                )
        return "\n".join(usage)

    def fetch_command(self, subcommand):
        """Use lit_commands version if it exists, otherwise fallback to django commands"""
        try: # find a lit_command version of the command
            return lit_commands[subcommand]
        except KeyError:
            # Fall back to default django  if we have not defined a custom command
            return super().fetch_command(subcommand)

In [ ]:
#| export
def cli():
    """This is set as entrypoint of the litdjango command in the package's setup.py"""
    utility = LitManagementUtility(sys.argv)
    utility.execute()

In [ ]:
!litdjango startproject --help

usage: litdjango startproject [-h] [--template TEMPLATE]
                              [--extension EXTENSIONS] [--name FILES]
                              [--exclude [EXCLUDE]] [--version] [-v {0,1,2,3}]
                              [--settings SETTINGS] [--pythonpath PYTHONPATH]
                              [--traceback] [--no-color] [--force-color]
                              name [directory]

Creates a Django project directory structure for the given project name in the
current directory or optionally in the given directory.

positional arguments:
  name                  Name of the application or project.
  directory             Optional destination directory

options:
  -h, --help            show this help message and exit
  --template TEMPLATE   The path or URL to load the template from.
  --extension EXTENSIONS, -e EXTENSIONS
                        The file extension(s) to render (default: "py").
                        Separate multiple extensions with commas, or use -e


<!--  -->